# Test Python Client

In [1]:
from src.ga_api_package.ga_api_client import System, Repository, HyperLogLog, pprint
import logging
logging.basicConfig(format='▸ %(asctime)s %(filename)s:%(lineno)d %(levelname)s %(message)s', level=logging.INFO)

## System

In [ ]:
system = System('https://rdlab-214.genie-analytics.com:8443/api', 'api', 'api123!@#')

In [ ]:
system_desc = await system.get_system()
pprint(system_desc)

In [ ]:
system_conf = await system.set_system(system_desc['config'])
pprint(system_conf)

In [ ]:
pprint(await system.get_all_tenants())

In [ ]:
tenant_desc = await system.create_tenant(0, {
    "limit":10000000000
})
pprint(tenant_desc)

In [ ]:
tenant_conf = await system.update_tenant(0, tenant_desc['config'])
pprint(tenant_conf)

In [ ]:
tenant_conf = await system.get_tenant(0)
pprint(tenant_conf)

In [ ]:
# await system.delete_tenant(0)

## Repository

In [2]:
repo = Repository('https://rdlab-214.genie-analytics.com/api', 'api@default', 'api123!@#')

## Using Dataset

In [3]:
# delete all datasets to have a clean environment
await repo.del_all_datasets()

In [4]:
dset_id = 13579
dset_conf = {
    'name':'example2',
    'series':'hour',
    'freq':300,
    'retainDepth':86400*7,
    'retainSize':10000000,
    'run':False
}
# create dataset
dset = await repo.set_dataset(dset_id, dset_conf)
# update dataset
dset = await repo.set_dataset(dset_id, dset_conf)
# get dataset
dset = await repo.get_dataset(dset_id)

In [ ]:
pipe_conf = {
    'name':'example',
    'pipeline': {
        'scope': {
            'i-field': '@controller.home',
            'i-entry': [
                [True,'t:0:0:1',0]
            ],
            'mode': '%directional'
        },
        'metric': [
            ['$sum', {
                'field':'@flow.bytes'
            }]
        ]
    }
}
# create pipeline
pipe = await dset.set_pipeline(1001, pipe_conf)
# update pipeline
pipe = await dset.set_pipeline(1001, pipe_conf)
# get pipeline
pipe = await dset.get_pipeline(1001)

In [ ]:
# patch dataset data
print(await dset.patch_data('2022-03-11 12:00:00+08:00', '1H'))

In [ ]:
# poll dataset data
print(await dset.poll_data('2022-03-11 12:00:00+08:00', '1H'))

In [ ]:
# query dataset data
df = await pipe.query_data('2022-03-11 12:00:00+08:00', '1H')
# df.columns = ['ts','forward','opposite']
print(df)

In [ ]:
# delete pipeline
await dset.del_pipeline(1001)

In [ ]:
# delete all pipelines
await dset.del_all_pipelines()

In [3]:
# delete dataset
await repo.del_dataset(dset_id)

In [ ]:
# delete all datasets
await repo.del_all_datasets()

### HyperLogLog

In [ ]:
adhoc = await repo.set_adhoc(300, {
    'scope': {
        'i-field': '@controller.home',
        'i-entry': [
            [True,'t:0:0:1',0]
        ],
        'mode': '%directional'
    },
    'metric': [
        ['$distinct', {
            'fields':['@flow.addr.dst']
        }]
    ]
})
df = await adhoc.query_data('2022-03-11 00:00:00', '1H', 'hour')
df.columns = ['ts','forward','opposite']
print(df)

In [ ]:
df2 = df.set_index('ts').sum()
df2

In [ ]:
df2.apply(HyperLogLog.value)

In [ ]:
await repo.close()